Create thumbnail images for testing

In [1]:
!pip install medmnist

  Obtaining dependency information for medmnist from https://files.pythonhosted.org/packages/ed/48/35b374bd8b9c6532e775bf89356d50aefe26c4383205f01a02e1995a9e2d/medmnist-3.0.0-py3-none-any.whl.metadata
  Using cached fire-0.5.0-py2.py3-none-any.whl
  Obtaining dependency information for termcolor from https://files.pythonhosted.org/packages/d9/5f/8c716e47b3a50cbd7c146f45881e11d9414def768b7cd9c5e6650ec2a80a/termcolor-2.4.0-py3-none-any.whl.metadata
Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)


In [6]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch
from torchvision import datasets, models
from torchvision import transforms as T
import torchvision.transforms.functional as TF
import random
import matplotlib.pyplot as plt

n_classes 8


In [7]:
info

{'python_class': 'BloodMNIST',
 'description': 'The BloodMNIST is based on a dataset of individual normal cells, captured from individuals without infection, hematologic or oncologic disease and free of any pharmacologic treatment at the moment of blood collection. It contains a total of 17,092 images and is organized into 8 classes. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images with resolution 3×360×363 pixels are center-cropped into 3×200×200, and then resized into 3×28×28.',
 'url': 'https://zenodo.org/records/10519652/files/bloodmnist.npz?download=1',
 'MD5': '7053d0359d879ad8a5505303e11de1dc',
 'url_64': 'https://zenodo.org/records/10519652/files/bloodmnist_64.npz?download=1',
 'MD5_64': '2b94928a2ae4916078ca51e05b6b800b',
 'url_128': 'https://zenodo.org/records/10519652/files/bloodmnist_128.npz?download=1',
 'MD5_128': 'adace1e0ed228fccda1f39692059dd4c',
 'url_224': 'https://zenodo.org/records/10519652/files/bloodmnist

# Customize a dataset
create a class for the customized dataset

In [10]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

class fundusisnotfun(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

Dataset preparation

We seperate the left/right eyes as two samples

In [31]:
from packages.small_batch_images import *
df = load_annotation()
df_healthy = df[df.N == 1]
labels = pd.DataFrame(columns=['image', 'age'])
# Seperate the each healthy sample into two samples
i = 0
for ind in df_healthy.index:
    row = df.loc[ind]
    for column in ['Left_Fundus', 'Right_Fundus']:
        labels.loc[i] = {'image': row[column], 'age': row['Age']}
        i+=1
# save as csv file
labels.to_csv('labels.csv')